#Chat with Your Data

The goal of this notebook is to conduct experiments that lay the groundwork for developing a web application. This application will allow users to interact with a database by posing natural language questions about the data it contains. This capability will enable users to easily access and retrieve information without needing to know complex database query languages.

The notebook is divided into several modules, each designed to guide you through different aspects of the process. Here’s a breakdown of what each part covers:

1. Setup and Installation: This initial section helps you install all the necessary libraries, set up the API, and prepare the database. For this workshop, we will be using SQLite and the sample database from Chinook. You can familiarize yourself with the database structure by clicking on this [Chinook Database link](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)
).

2. Understanding Agents: Here, we explore agents, which are special components of LangChain. These agents use large language models (LLMs) combined with a chain-of-thoughts to select and execute actions.

3. Crafting Chains: To get a deeper understanding of the underlying logic, we will create step-by-step sequences of commands using LangChain chains.

4. Managing Large Databases: The fourth section provides instructions on how to manage large databases that contain multiple tables. This will help you handle complex data structures efficiently.

5. Evaluation Principles: In the final module, we discuss the general principles for evaluating retrieval generation. We focus on how to use structured data as an external source of information to enhance our outcomes.

Each module is crafted to provide a thorough understanding and hands-on experience with practical tasks and concepts.


## Setups and Installation

In [ ]:
!pip install --upgrade --quiet  langchain langchain-community langchain-openai

### DB Setup

In this workshop, we'll explore the Chinook database, a sample database built with relational integrity. The Chinook database has relatively complex structure, so we can try to challenge the LLM with more complicated SQL queries.

In [ ]:
%load_ext sql

Create the empty database with name Chinook

In [ ]:
%%sql
sqlite:///Chinook.db

Download database content

In [ ]:
!wget https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite

--2024-04-22 07:47:09--  https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite [following]
--2024-04-22 07:47:09--  https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1067008 (1.0M) [application/octet-stream]
Saving to: ‘Chinook_Sqlite.sqlite’

Chinook_Sqlite.sqli 100%[===================>]   1.02M  --.-KB/

Check if there is some data

In [ ]:
%%sql sqlite:///Chinook_Sqlite.sqlite
SELECT * FROM Album limit 5

Done.


AlbumId,Title,ArtistId
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3


In [ ]:
!mv Chinook_Sqlite.sqlite Chinook.db

Construct an SQLAlchemy engine from URI. This allows us to use the database as a part of chains and agents.



In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [ ]:
db.get_usable_table_names()

['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']

### Connect to LLM

OpenAI's introduction of their chatbot was a pioneering success in the LLM applications. Despite many successors, OpenAI's tools and models continue to deliver more stable results, particularly when working with structured data. In this workshop, we will be using the GPT-3.5 models.  

Run the cell bellow and enter your OpenAI API key into the interactive window

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Agents
Agents are LangChain components that employ a language model to decide on a sequence of actions. While in Chains, a sequence of actions is explicitly predefined in the code, agents use a language model as a reasoning engine. This allows them to determine not only which actions to undertake but also the order in which to execute them.

The easiest way how to create our Chat with Data is to use an agent.

NOTE: Here we use OpenAI tools. For interaction with another LLM, we need to create a custom agent.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent

Note: we initiate the LLM with explicitly set temperature = 0. Temperature is a parameter of LLM that regulates randomness of the generated text. Higher number, higher randomness. To work with data and get predictable result, we need to limit the model creativity.

In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
agent_executor.invoke(
    "How many unique albums are there"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Album'}`



CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/
Invoking: `sql_db_query` with `{'query': 'SELECT COUNT(DISTINCT Title) AS UniqueAlbums FROM Album'}`


[(347,)]There are 347 unique albums in the database.

> Finished chain.


{'input': 'How many unique albums are there',
 'output': 'There are 347 unique albums in the database.'}

In [ ]:
agent_executor.invoke(
    "Which artist has the most albums?"
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Invoking: `sql_db_schema` with `{'table_names': 'Album, Artist'}`



CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Invoking: `sql_db_query` with `{'query': 'SELECT Artist.Name, COUNT(Album.AlbumId) AS AlbumCount FROM Artist JOIN Album ON Artist.ArtistId = Album.ArtistId GROUP BY Artist.Name ORDER BY AlbumCount DESC LIM

{'input': 'Which artist has the most albums?',
 'output': 'The artist with the most albums is Iron Maiden, with a total of 21 albums.'}

⭐ Congratulations! You have your own Chat with Data

### Custom Agent

Now let's create our own agent to see in details what's going on under the hood of the agents.

In [ ]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI # replace with anthropic
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

With custom agent we can use an LLM that supports Tool calling [beta]: Anthropic, Google Gemini, and Mistral.

One of the important component of the agent is SQLDatabaseToolkit, tools for interacting with an SQL database. Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again.

In [ ]:
custom_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [ ]:
custom_executor.run(
    "What artist has the most albums?"
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQL Agent Executor chain...
I need to find the artist with the most albums in the database.
Action: sql_db_query_checker
Action Input: SELECT artist, COUNT(album_id) AS num_albums FROM albums GROUP BY artist ORDER BY num_albums DESC LIMIT 1;SELECT artist, COUNT(album_id) AS num_albums 
FROM albums 
GROUP BY artist 
ORDER BY num_albums DESC 
LIMIT 1;I need to execute the query to get the artist with the most albums.
Action: sql_db_query
Action Input: SELECT artist, COUNT(album_id) AS num_albums FROM albums GROUP BY artist ORDER BY num_albums DESC LIMIT 1;Error: (sqlite3.OperationalError) no such table: albums
[SQL: SELECT artist, COUNT(album_id) AS num_albums FROM albums GROUP BY artist ORDER BY num_albums DESC LIMIT 1;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)I should check the list of tables in the database to see if there is an 'albums' table.
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, 

'Iron Maiden has the most albums.'

## Chains
 Chains are structured sequence of actions or operations that are pre-defined and coded into the application. These sequences are set up to handle specific tasks or processes. Unlike agents, which dynamically determine actions based on the reasoning provided by a language model, chains have a fixed sequence of actions that do not change based on input or conditions during runtime.


In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

Let's start with a simple chain that connects LLM with database.

In [ ]:
chain = create_sql_query_chain(llm, db)

First, generate an SQL query from user's question

In [ ]:
response = chain.invoke({"question": "How many artists are there"})
response

'SELECT COUNT("ArtistId") AS "TotalArtists" FROM "Artist"\nLIMIT 1;'

And try to run it in the database and get the answer

In [ ]:
db.run(response)

'[(275,)]'

Now let's create a chain that executes the query

In [ ]:
execute_query = QuerySQLDataBaseTool(db=db)

# here you can add your prompt as an argument
write_query = create_sql_query_chain(llm, db)

query_chain = write_query | execute_query
query_chain.invoke({"question": "How many artists are there"})

'[(275,)]'

Then create the chain to get the human-readable answer

In [ ]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
        """ Given the following user question, corresponding SQL query, and SQL result, answer the user question.

        Question: {question}
        SQL Query: {query}
        SQL Result: {result}
        Answer: """
)


answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many artists are there?"})

'There are 275 artists in the database.'

You can notice StrOutputParser() at the end of the chain. This component allows to get output in specific format, i.e. JSON.

##Dealing with large databases

In basically any SQL chain, we’ll need to feed the model at least part of the database schema. Without this it won’t be able to write valid queries. If the database has multiple tables, it could not fit to the context window.
What we can do in such cases is first extract the names of the tables related to the user input, and then include only their schemas.

One easy and reliable way to do this is using OpenAI function-calling and pydantic models.


In [ ]:
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field

###Table definitions and example rows
Our database comes with some convenience methods to give us the relevant context. Specifically, we can get the table names, their schemas, and a sample of rows from each table:

In [ ]:
context = db.get_context()
print(list(context))
print(context["table_info"])

['table_info', 'table_names']

CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("S

In [ ]:
class Table(BaseModel):
    #Table in SQL database.
    name: str = Field(description="Name of table in SQL database.")


table_names = "\n".join(db.get_usable_table_names())
system = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \

The tables are:

{table_names}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=system)
table_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

[Table(name='Genre'), Table(name='Artist'), Table(name='Track')]

In [ ]:
system = """Return the names of the SQL tables that are relevant to the user question. \
The tables are:

Music
Business"""
category_chain = create_extraction_chain_pydantic(Table, llm, system_message=system)
category_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

[Table(name='Music'), Table(name='Business')]

In [ ]:
from typing import List


def get_tables(categories: List[Table]) -> List[str]:
    tables = []
    for category in categories:
        if category.name == "Music":
            tables.extend(
                [
                    "Album",
                    "Artist",
                    "Genre",
                    "MediaType",
                    "Playlist",
                    "PlaylistTrack",
                    "Track",
                ]
            )
        elif category.name == "Business":
            tables.extend(["Customer", "Employee", "Invoice", "InvoiceLine"])
    return tables


table_chain = category_chain | get_tables
table_chain.invoke({"input": "What are all the genres of Alanis Morisette songs"})

['Album',
 'Artist',
 'Genre',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track',
 'Customer',
 'Employee',
 'Invoice',
 'InvoiceLine']

Now that we’ve got a chain that can output the relevant tables for any query we can combine this with our create_sql_query_chain, which can accept a list of table_names_to_use to determine which table schemas are included in the prompt:

In [ ]:
query_chain = create_sql_query_chain(llm, db)
# Convert "question" key to the "input" key expected by current table_chain.
table_chain = {"input": itemgetter("question")} | table_chain
# Set table_names_to_use using table_chain.
full_chain = RunnablePassthrough.assign(table_names_to_use=table_chain) | query_chain

In [ ]:
query = full_chain.invoke(
    {"question": "What are all the genres of Alanis Morisette songs"}
)
print(query)

SELECT DISTINCT "Genre"."Name"
FROM "Track"
JOIN "Album" ON "Track"."AlbumId" = "Album"."AlbumId"
JOIN "Artist" ON "Album"."ArtistId" = "Artist"."ArtistId"
JOIN "Genre" ON "Track"."GenreId" = "Genre"."GenreId"
WHERE "Artist"."Name" = 'Alanis Morissette'
LIMIT 5;


In [ ]:
db.run(query)

"[('Rock',)]"

Add the answer chain to get the nice result.

In [ ]:
answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [ ]:
chain.invoke(
    {"question": "What are all the genres of Alanis Morisette songs"}
)

'The genre of Alanis Morissette songs in the database is Rock.'

Voilà! In conclusion, we can say that chains allow you to construct logic much like building with LEGO: gather all the necessary details and assemble them in the correct order to achieve impressive results.

## Evaluation

We can apply RAG evaluation triade to Chat with SQL. Validation includes the three steps:
- Query aligns with user's question and guarantees correct data retrieval from the database
- Generated complition answers the question
- Generated answer contains information from the database.



###Query validation

langchain.agents module contains SQLDatabaseToolkit that designed to check query correctness. So agents already cover this part.
If you use chains, there are plenty ways how to utilize LLMs to validate the query. For instance, we can compare generated query with output of another LLM. To get measurable result, I suggest to use ROUGE score.

In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=13aa45f6289b4334463301abfda443d0df2c46ea9153617a5c78da29aa671c4f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

rouge_scores = scorer.score(query1, query2)
rouge_scores

We expect this nuber is close to 1, since the query should be identical.

### Complition validation

BERT-score (similarity between question and answer).
First, we need to vectorize question and answer. Here we use sentence-transformer library with pretrained embedding model. It's a python framework that allow to get a vector representation of the whole sentence (not separate tokens). Under the hood, it has Sentence-BERT model, the Seamise Network that creates representation vectors based on similarity of two sentences. Then we calculate cosine similarity between the user's question and generated answer.  

In [ ]:
! pip install -q sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

In [ ]:
embedded_question = model.encode(question)
embedded_answer = mode.encode(answer)

bert_score = util.pytorch_cos_sim(embedded_question, embedded_answer)[0].tolist()
bert_score

Today, there are several tools available for evaluating application powered by LLMs, which themselves are based on LLMs. For example, Galileo, TruLens, and LangChain's evaluation modules. I recommend you to try these resources, if you want to include evaluation in the work flow.  

##Conclusion
As we wrap up our workshop, it's clear that the tools and techniques we've explored change the way we handle and interact with data. By utilizing the principles of LangChain, we've learned to integrate LLMs into applications, enabling us to manage databases and respond to queries with unprecedented ease and efficiency.

Thank you for your participation and curiosity. We encourage you to continue experimenting with the skills and knowledge gained to empower your work with LLMs.